# PyTorch model on Spark using SparkTorch

In this notebook, which is supposed to run within a Databricks cluster, we present some sample code for the training and evaluation of a Torch model on the distributed engine provided by Spark using the PySpark API. The model is rather trivial, because emphasis is given on how we retrieve the data from a Blob Storage (for example, within Microsoft's Azure Data Lake Gen 2), pre-process it using Spark and then train and evaluate a PyTorch model on the distributed data.

In [0]:
%pip install sparktorch

Python interpreter will be restarted.
  Created wheel for sparktorch: filename=sparktorch-0.1.2-py3-none-any.whl size=24597 sha256=4f0d08d913b9b4b30f44b75c484f70453db07a8cbc911df3eadd347f78dff413
  Stored in directory: /root/.cache/pip/wheels/3a/45/cf/a35fb54abecf5880c8da29d74011a2ab3a1d6530521f5c970c
Successfully built sparktorch
Python interpreter will be restarted.


In [0]:
# A secret scope is defined so that these keys are drawn from a keyvault
# instead of being hard-copied here
dlname = dbutils.secrets.get(scope = "testtuts", key = "lakename")
service_credential = dbutils.secrets.get(scope="testtuts", key="adcredential")
application_id = dbutils.secrets.get(scope="testtuts", key="DBappID")
directory_id = dbutils.secrets.get(scope="testtuts", key="DBtenantID")

# Establish connection with Blob Storage
spark.conf.set("fs.azure.account.auth.type."+dlname+".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type."+dlname+".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id."+dlname+".dfs.core.windows.net", application_id)
spark.conf.set("fs.azure.account.oauth2.client.secret."+dlname+".dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint."+dlname+".dfs.core.windows.net", "https://login.microsoftonline.com/"+directory_id+"/oauth2/token")

# container containing the datafiles
containername = "torchtut"

# Train dataset
train_data_url = f"abfss://{containername}@{dlname}.dfs.core.windows.net/train.csv"
traindf = spark.read.format("csv").load(train_data_url, inferSchema = True, header = True)

# Test dataset
test_data_url = f"abfss://{containername}@{dlname}.dfs.core.windows.net/test.csv"
testdf = spark.read.format("csv").load(test_data_url, inferSchema = True, header = True)

The dataset is the famous MNIST handwritten digits dataset, with 60000 train samples and 10000 test samples. Each sample consists of 28x28 pixel values named `pixelx`, where x is any value from 1 to 28x28, as well as an identification (label). More realistic applications will make use of more complex and/or larger datasets, however this suffices for the purposes of this notebook. The next cell's output corresponds to a random collection of train samples from the `traindf` spark dataframe.

In [0]:
display(traindf.limit(3))

pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pixel84,pixel85,pixel86,pixel87,pixel88,pixel89,pixel90,pixel91,pixel92,pixel93,pixel94,pixel95,pixel96,pixel97,pixel98,pixel99,pixel100,pixel101,pixel102,pixel103,pixel104,pixel105,pixel106,pixel107,pixel108,pixel109,pixel110,pixel111,pixel112,pixel113,pixel114,pixel115,pixel116,pixel117,pixel118,pixel119,pixel120,pixel121,pixel122,pixel123,pixel124,pixel125,pixel126,pixel127,pixel128,pixel129,pixel130,pixel131,pixel132,pixel133,pixel134,pixel135,pixel136,pixel137,pixel138,pixel139,pixel140,pixel141,pixel142,pixel143,pixel144,pixel145,pixel146,pixel147,pixel148,pixel149,pixel150,pixel151,pixel152,pixel153,pixel154,pixel155,pixel156,pixel157,pixel158,pixel159,pixel160,pixel161,pixel162,pixel163,pixel164,pixel165,pixel166,pixel167,pixel168,pixel169,pixel170,pixel171,pixel172,pixel173,pixel174,pixel175,pixel176,pixel177,pixel178,pixel179,pixel180,pixel181,pixel182,pixel183,pixel184,pixel185,pixel186,pixel187,pixel188,pixel189,pixel190,pixel191,pixel192,pixel193,pixel194,pixel195,pixel196,pixel197,pixel198,pixel199,pixel200,pixel201,pixel202,pixel203,pixel204,pixel205,pixel206,pixel207,pixel208,pixel209,pixel210,pixel211,pixel212,pixel213,pixel214,pixel215,pixel216,pixel217,pixel218,pixel219,pixel220,pixel221,pixel222,pixel223,pixel224,pixel225,pixel226,pixel227,pixel228,pixel229,pixel230,pixel231,pixel232,pixel233,pixel234,pixel235,pixel236,pixel237,pixel238,pixel239,pixel240,pixel241,pixel242,pixel243,pixel244,pixel245,pixel246,pixel247,pixel248,pixel249,pixel250,pixel251,pixel252,pixel253,pixel254,pixel255,pixel256,pixel257,pixel258,pixel259,pixel260,pixel261,pixel262,pixel263,pixel264,pixel265,pixel266,pixel267,pixel268,pixel269,pixel270,pixel271,pixel272,pixel273,pixel274,pixel275,pixel276,pixel277,pixel278,pixel279,pixel280,pixel281,pixel282,pixel283,pixel284,pixel285,pixel286,pixel287,pixel288,pixel289,pixel290,pixel291,pixel292,pixel293,pixel294,pixel295,pixel296,pixel297,pixel298,pixel299,pixel300,pixel301,pixel302,pixel303,pixel304,pixel305,pixel306,pixel307,pixel308,pixel309,pixel310,pixel311,pixel312,pixel313,pixel314,pixel315,pixel316,pixel317,pixel318,pixel319,pixel320,pixel321,pixel322,pixel323,pixel324,pixel325,pixel326,pixel327,pixel328,pixel329,pixel330,pixel331,pixel332,pixel333,pixel334,pixel335,pixel336,pixel337,pixel338,pixel339,pixel340,pixel341,pixel342,pixel343,pixel344,pixel345,pixel346,pixel347,pixel348,pixel349,pixel350,pixel351,pixel352,pixel353,pixel354,pixel355,pixel356,pixel357,pixel358,pixel359,pixel360,pixel361,pixel362,pixel363,pixel364,pixel365,pixel366,pixel367,pixel368,pixel369,pixel370,pixel371,pixel372,pixel373,pixel374,pixel375,pixel376,pixel377,pixel378,pixel379,pixel380,pixel381,pixel382,pixel383,pixel384,pixel385,pixel386,pixel387,pixel388,pixel389,pixel390,pixel391,pixel392,pixel393,pixel394,pixel395,pixel396,pixel397,pixel398,pixel399,pixel400,pixel401,pixel402,pixel403,pixel404,pixel405,pixel406,pixel407,pixel408,pixel409,pixel410,pixel411,pixel412,pixel413,pixel414,pixel415,pixel416,pixel417,pixel418,pixel419,pixel420,pixel421,pixel422,pixel423,pixel424,pixel425,pixel426,pixel427,pixel428,pixel429,pixel430,pixel431,pixel432,pixel433,pixel434,pixel435,pixel436,pixel437,pixel438,pixel439,pixel440,pixel441,pixel442,pixel443,pixel444,pixel445,pixel446,pixel447,pixel448,pixel449,pixel450,pixel451,pixel452,pixel453,pixel454,pixel455,pixel456,pixe

Next, we need to perform any pre-processing necessary. In this case a simple min-max scaling will suffice, however in general each column may need its own unique pre-processing approach for feature engineering. Here, we first combine all features (pixels) into a single feature vector using the `VectorAssembler` and perform min-max scaling in this vector, since it is faster than performing it in each column separately.

In [0]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

columns_for_minmax = [col for col in traindf.columns if col != "label"]

# Define a vector assembler from each column to be normalized
assembler = VectorAssembler(inputCols=columns_for_minmax, outputCol="features")

# Define the corresponding scaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

Note that in the steps above we simply defined the assembler and scaler, without transforming anything. This is because we intend to incorporate all of these steps in a common pipeline which takes care of all transformations plus model training, a bit further down the road. Now, we may move on with defining the Torch model which will be used. The model is a CNN network with linear layers at its end. Note that due to peculiarities of the SparkTorch library (see [this](https://github.com/dmmiller612/sparktorch/issues/22) issue for example), the network needs to be saved in a separate file available in the python path. This is why the class defining the CNN Model is not part of this notebook, but of the separate file `NNcode.py` which is uploaded locally to the DBFS.

In [0]:
import torch
import torch.nn as nn

# Upload the NNcode.py file into the dbfs (or choose another way of doing this)
# and then inform spark to add the file
spark.sparkContext.addPyFile("dbfs:/FileStore/shared_uploads/NNcode.py")

# Now the file is recognized and the relevant class can be imported
from NNcode import CNNModel

For the purposes of this presentation, we will use a CNN with 4 convolutional layers and 3 linear layers. The settings shown below define the used model.

In [0]:
import numpy as np

input_height = int(np.sqrt(len(traindf.columns)-1))
input_width = input_height

conv_channels = [1,4,16,32,64]
kernels = [3,3,3,3]
maxpools = [2,2,2,2]
lin_channels = [128,64,10]
dropout = 0.1

model = CNNModel(input_height = input_height, input_width = input_width,
                    conv_channels = conv_channels, kernels = kernels, maxpools = maxpools,
                    lin_channels = lin_channels, dropout = dropout)

With the model defined, we proceed by importing the `SparkTorch` module which acts as the bridge between PyTorch and Spark. We first build a serialized torch object (think of it as a collection of model, loss function and optimizer) and then define the SparkTorch model using this serialized object, along with other training parameters.

In [0]:
from sparktorch import serialize_torch_obj, SparkTorch

# Parameters
learning_rate = 0.001

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

# Build the CNN as a serialized torch object
torch_obj = serialize_torch_obj(model=model, criterion=loss_function, optimizer=optimizer, lr=learning_rate)

In [0]:
# Here we define the SparkTorch model and the rest of the training's parameters
spark_cnn = SparkTorch(
        inputCol = 'scaledFeatures',
        labelCol = 'label',
        predictionCol = 'predictions',
        torchObj = torch_obj,
        iters = 150,
        verbose = 2,
        validationPct = 0.3,
        earlyStopPatience = 20,
        miniBatch = 300
    )

The parameters used in the previous cell correspond to the following:
- `inputCol`: The dataframe's column holding the feature vector that is the model's input
- `labelCol`: The dataframe's column holding the label corresponding to the feature vector.
- `predictionCol`: A new column in the dataframe to hold the predictions of the model.
- `torchObj`: The serialized torch object (including the model, the optimizer and the loss function).
- `iters`: The number of training epochs.
- `validationPct`: The percentage of data to be used for validation.
- `earlyStopPatience`: If using validation data, the number of patience steps before early stopping is activated.
- `miniBatch`: Mini-batch size for training.
- `verbose`: A check of whether to log the losses or not.

We are now ready to define our full pipeline, including the vector assembler, the scaler, as well as the training of the SparkTorch model and its subsequent evaluation.

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Merge all 3 steps into a pipeline
pipeline = Pipeline(stages=[assembler, scaler, spark_cnn])

# Fit the pipeline on the training data, i.e. transform the traindf properly to
# acquire the scaled feature vector and then train the CNN model
pipeline = pipeline.fit(traindf)

# Finally, we apply the same pipeline on the test data, in order to acquire the predictions
# and calculate the accuracy of our model.
predictions = pipeline.transform(testdf).persist()

# Run predictions and evaluation
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="predictions", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

print(f"The trained model's accuracy on the evaluation set is {accuracy*100:.3f}%.")

The trained model's accuracy on the evaluation set is 92.810%.


In [0]:
# Note that if we wanted to save the model after training so that it could be loaded at another time
# for inference, we would save the pipeline by writing:
# pipeline.write().overwrite().save('cnnmodel')
# Once saved, this pipeline could be loaded as
# loaded_pipeline = PysparkPipelineWrapper.unwrap(PipelineModel.load('cnnmodel'))